In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import time

import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import files
uploaded = files.upload()

Saving evgenyi_onegin.txt to evgenyi_onegin (1).txt


In [3]:
text = open('evgenyi_onegin.txt','rb').read().decode(encoding = 'utf-8')
print('Length of text: {} characters'.format(len(text)))

Length of text: 286984 characters


In [4]:
print(text[:200])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                      


In [5]:
vocab = sorted(set(text))
print(f'{len(vocab)} уникальных символов в тексте.')

131 уникальных символов в тексте.


In [6]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [7]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [8]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [9]:
BATCH_SIZE = 64

EPOCHS = 20

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [10]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [11]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units // 2,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units * 2,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [12]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           33536     
                                                                 
 lstm (LSTM)                 (64, None, 512)           1574912   
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          6295552   
                                                                 
 lstm_2 (LSTM)               (64, None, 2048)          25174016  
                                                                 
 lstm_3 (LSTM)               (64, None, 1024)          12587008  
                                                                 
 dense (Dense)               (64, None, 131)           134275    
                                                                 
Total params: 45,799,299
Trainable params: 45,799,299
No

In [13]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
    
model.compile(optimizer='adam', loss=loss)

In [14]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*5,
    save_weights_only=True)

In [15]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
44/44 [==============================] - 33s 525ms/step - loss: 2.3532
Epoch 2/20
44/44 [==============================] - 25s 557ms/step - loss: 1.7805
Epoch 3/20
44/44 [==============================] - 25s 556ms/step - loss: 1.6047
Epoch 4/20
44/44 [==============================] - 24s 545ms/step - loss: 1.5045
Epoch 5/20
44/44 [==============================] - 25s 568ms/step - loss: 1.4433
Epoch 6/20
44/44 [==============================] - 24s 546ms/step - loss: 1.3781
Epoch 7/20
44/44 [==============================] - 25s 565ms/step - loss: 1.3462
Epoch 8/20
44/44 [==============================] - 25s 554ms/step - loss: 1.3163
Epoch 9/20
44/44 [==============================] - 24s 547ms/step - loss: 1.2970
Epoch 10/20
44/44 [==============================] - 27s 610ms/step - loss: 1.2912
Epoch 11/20
44/44 [==============================] - 25s 556ms/step - loss: 1.2497
Epoch 12/20
44/44 [==============================] - 25s 556ms/step - loss: 1.2219
Epoch 13/20
4

In [16]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [17]:
def generate_text(model, start_string):
    num_generate = 700

    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    temperature = 0.8

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [18]:
start_string = "Хотел бы я тебе представить"
text_ = generate_text(model, start_string=start_string)
print(text_)

Хотел бы я тебе представить, модет, чемана,
                                                                    XXI

                        В его и печально одна.

                          С немалих обо свою стони,
                        Госпешь мой геное тома:
                                               Былки гордала ждет встрение
                       Ни отпидался в одно, я вот вад уна.
                         Подада драдалых окорой
                        Вот модки мне в лешине веда
                           Все токе даже на читьени!
                             Содоюну подавать брека;
                                                XXXIII

                        Он безумный выстрого мог {2}
             


Модель снизила loss, но модель все - равно нуждается в улучшениях